# Resume-to-Job Gap Analysis Tool

### **Project Summary**
This project demonstrates the use of a Large Language Model (LLM) to perform a sophisticated analysis task with real-world business value. The tool automates the tedious process of manually comparing a candidate's resume against a job description. By providing a job description URL and a candidate's resume text, this notebook generates a detailed cover letter and "gap analysis" report. This report highlights which skills are matched, which are missing, and provides an overall suitability score, enabling recruiters to screen candidates more efficiently and helping applicants tailor their resumes effectively.

### **How to Use**
1.  **Set up your Environment**: Make sure you have a `.env` file in the root directory with your `OPENAI_API_KEY`.
2.  **Input the Job URL**: In **Section 2**, paste the URL of a web-based job description into the `job_description_url` variable.
3.  **Input the Resume**: In **Section 2**, paste the candidate's full resume text into the `resume_text` variable.
4.  **Run the Notebook**: Execute the cells from top to bottom. The final cell in **Section 6** will display the formatted analysis report.

### **A Note on Ethical Web Scraping**
This tool uses the `requests` library to fetch website content. To ensure compliance and responsible use:
* We send a standard `User-Agent` header to identify our script as a web browser, which is a common practice for preventing being blocked.
* **Always be mindful of the website's terms of service.** Automated scraping may be disallowed on some sites. This tool is intended for educational purposes and should be used on publicly accessible job postings where such activity is permitted.

## 1. Setup:

In [ ]:
# Imports
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
# Load Environment Variables
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

#### Test OpenAI API Key

In [ ]:
# Validate API key
if not api_key:
    print("ERROR: No API key found - please add OPENAI_API_KEY to your .env file")
elif not api_key.startswith("sk-proj-"):
    print("WARNING: API key format may be incorrect")
elif api_key.strip() != api_key:
    print("ERROR: API key has whitespace - please remove extra spaces/tabs")
else:
    print("SUCCESS: API key loaded successfully")

# Initialize OpenAI client
openai = OpenAI()

## 2. Data Input

In [ ]:
# The URL for the Y Combinator job posting you want to analyze. (ycombinator.com/companies/y-combinator/jobs//jobs)
job_url = "https://www.ycombinator.com/companies/y-combinator/jobs/rq3DaTs-product-engineer"

# Replace this example resume with the actual candidate's resume text.
resume_text = """
John Doe
123 Main Street, Anytown, USA | (123) 456-7890 | john.doe@email.com

Summary
Software Engineer with 5 years of experience in web applications. 
Proficient in Python and JavaScript with a strong background in AWS.

Experience
Senior Software Engineer | Tech Solutions Inc. | 2021 - Present
- Led development of analytics dashboard using React and Python
- Architected microservices backend on AWS
- Mentored junior engineers

Software Engineer | Innovate Corp. | 2018 - 2021
- Developed e-commerce platform using Python and Django
- Wrote comprehensive unit and integration tests

Skills
Python, JavaScript, React, Flask, Django, AWS, Docker, Git
"""



## 3. Prompt Engineering

In [ ]:
SYSTEM_PROMPT = """
You are a strategic career advisor. Your task is to synthesize a candidate's resume and a job description into a compelling, two-part analysis. Your goal is to create a narrative connecting the candidate's specific accomplishments to the company's needs.

**Formatting:** Use markdown with bolding for emphasis. Do not use placeholders like '[Job Title]'; infer the details from the text.

---

# Part 1: Candidate Suitability Analysis

## Executive Summary
Provide a 2-3 sentence summary of the candidate's alignment with the role, stating your professional opinion on their potential.

## Key Strengths & Evidence
List the top 3 strengths the candidate brings. For each strength, **quote or paraphrase evidence directly from the resume's 'Experience' section**.
* **Strength:** [Example: Scalable Backend Development] - **Evidence:** [Example: "Architected microservices backend on AWS," demonstrating hands-on experience.]

## Areas for Growth & Discussion
Identify key requirements from the job description not explicitly covered in the resume. Frame these as **strategic points to address in an interview**.
* **Topic:** [Example: TypeScript Proficiency] - **Suggested Question:** "The role heavily uses TypeScript. Could you discuss your experience level with it and your approach to learning new languages?"

## Holistic Suitability Score
Provide a score (e.g., 85/100) and justify it in one sentence.

---

# Part 2: Dynamic Cover Letter Draft
Generate a compelling and authentic cover letter from the candidate's perspective.
"""

## 4. Webscraper

In [ ]:
# Scraper Function
def scrape_ycombinator_job(url: str) -> str:
    """
    Scrapes a single job posting from a ycombinator.com URL.

    Args:
        url: The URL of the Y Combinator job posting.

    Returns:
        The cleaned text of the job description, or an error message.
    """
    print(f"INFO: Attempting to scrape YC job posting from: {url}")
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    try:
        # Fetch the page content
        response = requests.get(url, headers=headers, timeout=10)
        # Raise an error if the page is not found (e.g., 404)
        response.raise_for_status()

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the job title (specifically from the <h1> tag)
        title_element = soup.select_one('h1')
        title = title_element.get_text(strip=True) if title_element else "Job Title Not Found"

        # Extract the main job description content (from the <div class="prose">)
        description_element = soup.select_one('.prose')
        description = description_element.get_text(separator='\n', strip=True) if description_element else ""
        
        # Combine them for the final text
        full_text = f"Job Title: {title}\n\n{description}"
        
        print("SUCCESS: Scraping complete.")
        return full_text

    except requests.exceptions.RequestException as e:
        print(f"ERROR: Scraping failed. Could not fetch URL. {e}")
        return "[Scraping failed: Could not connect to the server]"
    except Exception as e:
        print(f"ERROR: An unexpected error occurred during scraping: {e}")
        return "[Scraping failed: An unexpected error occurred]"


## 5. Gap Analysis

In [ ]:
def get_analysis(job_description: str, resume: str) -> str:
    """Sends the job description and resume to the AI and returns the analysis."""
    print("INFO: Sending data to the AI for analysis...")
    user_prompt = f"""Please generate the analysis based on the following documents.

    **JOB DESCRIPTION:**
    ---
    {job_description}
    ---

    **CANDIDATE RESUME:**
    ---
    {resume}
    ---
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    print("SUCCESS: Analysis complete.")
    return response.choices[0].message.content

## 6. Execution

In [ ]:
# Scrape the job description text from the URL
job_description_text = scrape_ycombinator_job(job_url)

# Only proceed if scraping was successful
if not job_description_text.startswith("[Scraping failed"):
    # Run the analysis with the scraped text
    analysis_report = get_analysis(job_description_text, resume_text)
    # Display the final report
    display(Markdown(analysis_report))
else:
    # If scraping failed, display the error message
    display(Markdown(f"## {job_description_text}"))